# Task 3

## Classification and Neural Network

### Importing Libraries and Data

##### Necessary libraries are imported for data analysis and machine learning, NBA rookie data is loaded from as CSV file, and first few rows of the dataset after removing any missing values are shown.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")


# Loading the NBA rookie data from the CSV file
data = pd.read_csv('C:/Users/Dell/Documents/GitHub/King-Deputy/Regression,Clustering,Classification& Neural Network Assignment/Task 3 Classification, Neural Network/nba_rookie_data.csv')
data = data.dropna()
print(data.head())

### Data Exploration and Processing

##### This tasks help in understanding the data, identifying missing values, checking data types, exploring summary statistics, and understanding the distribution of the target variable.

In [ ]:
# Checking for missing values in the dataset
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Checking the data types of each column
data_types = data.dtypes
print("Data Types:\n", data_types)

# Checking summary statistics of the dataset
summary_stats = data.describe()
print("Summary Statistics:\n", summary_stats)

# Checking the distribution of the target variable
target_distribution = data['TARGET_5Yrs'].value_counts()
print("Target Variable Distribution:\n", target_distribution)

In [ ]:
count_0 = data['TARGET_5Yrs'].value_counts()[0]
count_1 = data['TARGET_5Yrs'].value_counts()[1]

# Creating a bar chart for the target variable
plt.bar(['0', '1'], [count_0, count_1])
plt.xlabel('TARGET_5Yrs')
plt.ylabel('Count')
plt.title('Target Variable Distribution')
plt.show()

### Data Preprocessing

In [ ]:
# Seperating the features (X) and the target variable (y)
X = data.drop(columns=['TARGET_5Yrs', 'Name'])
y = data['TARGET_5Yrs']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


### Building and Evaluating Machine Learning Models

#### Using Logistic Regression

In [ ]:
# Using Logistic Regression
logistic_model = LogisticRegression(random_state=42)

# Fitting the model on the training data
logistic_model.fit(X_train, y_train)

y_pred_logistic = logistic_model.predict(X_test)

# Evaluating the Logistic Regression model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
confusion_matrix_logistic = confusion_matrix(y_test, y_pred_logistic)

print("Logistic Regression Model:")
print("Accuracy:", accuracy_logistic)
print("\nConfusion Matrix:\n", confusion_matrix_logistic)
print("\nClassification Report:\n", classification_report(y_test, y_pred_logistic))


##### The Odd ratio is computed to understand the impact of each feature on the odds of the target variable. Positive odds ratios indicate an increase in the odds of the target variable, while negative values suggest a decrease.

In [ ]:
# Getting odds ratios
odds_ratios = np.exp(logistic_model.coef_)

# Creating a DataFrame to display the odds ratios
odds_ratios_df = pd.DataFrame(odds_ratios, columns=X_train.columns, index=['Odds Ratio'])
print(odds_ratios_df)

In [ ]:
# Visualise the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_logistic, annot=True, fmt="d", cmap="Blues", linewidths=.5, square=True, cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Logistic Regression Confusion Matrix")
plt.show()


#### Using GaussianNB

In [ ]:
#Using GaussianNB
nb_model = GaussianNB()

# Fitting the model on the training data
nb_model.fit(X_train, y_train)

# Predicting using the Gaussian Naive Bayes model
y_pred_nb = nb_model.predict(X_test)

nb_accuracy = accuracy_score(y_test, y_pred_nb)
nb_precision = precision_score(y_test, y_pred_nb)
nb_recall = recall_score(y_test, y_pred_nb)

print("Gaussian Naive Bayes Model Metrics:")
print(f"Accuracy: {nb_accuracy:.2f}")
print(f"Precision: {nb_precision:.2f}")
print(f"Recall: {nb_recall:.2f}")


**These metrics provide an assessment of the Gaussian Naive Bayes model's performance on the test set, indicating its ability to correctly classify instances of each class.**

##### Summary of the class-conditional distribution for each class is provided, indicating the mean and standard deviation for each feature. It helps in understanding the distribution of feature values within each class

In [ ]:
# Displaying class-conditional distribution summary
for class_label in np.unique(y_train):
    class_indices = (y_train == class_label)
    class_data = X_train.loc[class_indices]

    # Using mean and std to get class-conditional distribution summary
    class_mean = class_data.mean()
    class_std = class_data.std()

    print(f"\nClass {class_label} - Mean:")
    print(class_mean)

    print(f"\nClass {class_label} - Standard Deviation:")
    print(class_std)

In [ ]:
# Generating the confusion matrix for GaussianNB
cm_nb = confusion_matrix(y_test, y_pred_nb)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Gaussian Naive Bayes')

plt.show()

#### Using Neural Network

In [ ]:
# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### The below code initializes, compiles, and trains a basic neural network (Sequential model) using Keras with TensorFlow backend. It then evaluates the model on the test set and prints accuracy, loss, and the confusion matrix.

In [ ]:
# Initializing a Sequential model
model = Sequential()

# Adding input and hidden layers
model.add(Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compiling
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fitting the model on the training data
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

# Evaluating the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)

print("Neural Network Model Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Loss: {loss:.2f}")

y_pred_nn = model.predict(X_test_scaled)

y_pred_nn_binary = (y_pred_nn > 0.5).astype(int)
nn_cm = confusion_matrix(y_test, y_pred_nn_binary)

##### The gradients of the binary crossentropy loss with respect to the trainable variables in the neural network model is computed. The gradients are then printed for each variable. The GradientTape is used to record operations for automatic differentiation.

In [ ]:
#Computing the gradients
import tensorflow as tf

# Convert y_test to a NumPy array and reshape it to have the same shape as predictions
y_test_array = y_test.to_numpy().reshape(predictions.shape)

with tf.GradientTape() as tape:
    predictions = model(X_test_scaled, training=False)
    loss = tf.keras.losses.binary_crossentropy(y_test_array, predictions)

gradients = tape.gradient(loss, model.trainable_variables)

# Printing the gradients
print("\nGradients with respect to the predictors:")
for variable, gradient in zip(model.trainable_variables, gradients):
    print(f"{variable.name}: {gradient.numpy()}")

In [ ]:
# Creating a heatmap to visualize the confusion matrix for Neural Network
plt.figure(figsize=(8, 6))
sns.heatmap(nn_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Neural Network')

plt.show()

### Comparing the models and Visualizing

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Calculating ROC curve and AUC for Logistic Regression
fpr_logistic, tpr_logistic, _ = roc_curve(y_test, y_pred_logistic)
roc_auc_logistic = roc_auc_score(y_test, y_pred_logistic)

# Calculating ROC curve and AUC for Gaussian Naive Bayes
fpr_nb, tpr_nb, _ = roc_curve(y_test, y_pred_nb)
roc_auc_nb = roc_auc_score(y_test, y_pred_nb)

# Calculating ROC curve and AUC for Neural Network
fpr_nn, tpr_nn, _ = roc_curve(y_test, y_pred_nn)
roc_auc_nn = roc_auc_score(y_test, y_pred_nn)

# Plotting ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_logistic, tpr_logistic, label='Logistic Regression (AUC = %0.2f)' % roc_auc_logistic)
plt.plot(fpr_nb, tpr_nb, label='Gaussian Naive Bayes (AUC = %0.2f)' % roc_auc_nb)
plt.plot(fpr_nn, tpr_nn, label='Neural Network (AUC = %0.2f)' % roc_auc_nn)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()


### Improving the Models by using Hyperparameter Tuning

#### Hyperparameter Tuning for Logistic Regression

In [ ]:
# Defining the hyperparameter grid for Logistic Regression for model improvement
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'max_iter': [100, 200, 300, 400]  # Maximum number of iterations
}

# Creating a Logistic Regression model
lr_model = LogisticRegression()

# Creating a GridSearchCV object for Logistic Regression
lr_grid = GridSearchCV(lr_model, param_grid_lr, cv=5, scoring='accuracy')

# Fitting the GridSearchCV object to ythe data
lr_grid.fit(X_train, y_train)

# Getting the best parameters and the best estimator
best_params_lr = lr_grid.best_params_
best_lr_model = lr_grid.best_estimator_

y_pred_Tuned_logistic = best_lr_model.predict(X_test)

# Evaluating the tuned Logistic Regression model
accuracy_logistic_tuned = accuracy_score(y_test, y_pred_Tuned_logistic)
confusion_matrix_logistic_tuned = confusion_matrix(y_test, y_pred_Tuned_logistic)

print("Tuned Logistic Regression Model:")
print("Accuracy:", accuracy_logistic_tuned)
print("\nConfusion Matrix:\n", confusion_matrix_logistic_tuned)
print("\nClassification Report:\n", classification_report(y_test, y_pred_Tuned_logistic))

# Visualising the Confusion Matrix for the tuned Logistic model
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix_logistic_tuned, annot=True, fmt="d", cmap="Blues", linewidths=.5, square=True, cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Tuned Logistic Regression Confusion Matrix")
plt.show()


In [ ]:
# Getting odds ratios for the Tuned Logistic Regression
odds_ratios_Tunedlr = np.exp(best_lr_model.coef_)

# Creating a DataFrame to display the odds ratios
odds_ratios_Tunedlr_df = pd.DataFrame(odds_ratios_Tunedlr, columns=X_train.columns, index=['Odds Ratio'])
print(odds_ratios_Tunedlr_df)

#### Hyperparameter Tuning for Neural Network

In [ ]:
# Defining the hyperparameter grid for Neural Network
param_grid_nn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # L2 regularization term
    'max_iter': [100, 200, 300, 400]  # Maximum number of iterations
}
# Creating a Neural Network model
nn_model = MLPClassifier()

# Creating a GridSearchCV object for Neural Network
nn_grid = GridSearchCV(nn_model, param_grid_nn, cv=5, scoring='accuracy')

# Fitting the GridSearchCV object to the data
nn_grid.fit(X_train_scaled, y_train)

# Getting the best parameters and the best estimator
best_params_nn = nn_grid.best_params_
best_nn_model = nn_grid.best_estimator_

y_pred_Tuned_nn = best_nn_model.predict(X_test_scaled)

# Evaluating the tuned neural network model
accuracy = accuracy_score(y_test, y_pred_Tuned_nn)
y_pred_nn_binary_Tuned = (y_pred_Tuned_nn > 0.5).astype(int)
nn_cm_tuned = confusion_matrix(y_test, y_pred_nn_binary_Tuned)

# Creating a heatmap to visualize the confusion matrix for the Tuned Neural Network
plt.figure(figsize=(8, 6))
sns.heatmap(nn_cm_tuned, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Tuned Neural Network')

plt.show()

### Comparing the Tuned Models

In [ ]:
# Calculating ROC curve and AUC for Tuned Logistic Regression
fpr_logistic_tuned, tpr_logistic_tuned, _ = roc_curve(y_test, y_pred_Tuned_logistic)
roc_auc_logistic_tuned = roc_auc_score(y_test, y_pred_Tuned_logistic)

# Calculating ROC curve and AUC for Gaussian Naive Bayes ( No Change)
fpr_nb, tpr_nb, _ = roc_curve(y_test, y_pred_nb)
roc_auc_nb = roc_auc_score(y_test, y_pred_nb)

# Calculating ROC curve and AUC for Neural Network
fpr_nn_tuned, tpr_nn_tuned, _ = roc_curve(y_test, y_pred_Tuned_nn)
roc_auc_nn_tuned = roc_auc_score(y_test, y_pred_Tuned_nn)

# Plotting ROC curves for the tuned model
plt.figure(figsize=(8, 6))
plt.plot(fpr_logistic_tuned, tpr_logistic_tuned, label='Tuned Logistic Regression (AUC = %0.2f)' % roc_auc_logistic_tuned)
plt.plot(fpr_nb, tpr_nb, label='Gaussian Naive Bayes (AUC = %0.2f)' % roc_auc_nb)
plt.plot(fpr_nn_tuned, tpr_nn_tuned, label='Tuned Neural Network (AUC = %0.2f)' % roc_auc_nn_tuned)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()